https://docs.google.com/spreadsheets/d/1wtC6knl3ICXd4wBEKfXk97vT0ixuRrS8/edit?usp=share_link&ouid=104311366367824774650&rtpof=true&sd=true

Название файла: sales_data.xlsx\
Описание данных:
- Product (Продукт): Название продукта, который был продан в интернет-магазине. Продукт может быть одним из следующих: Product A, Product B, Product C, Product D, Product E.
- Date (Дата): Дата продажи продукта.
- Quantity (Количество): Количество проданных продуктов.
- Revenue (Выручка): Сумма денег, полученная от продажи продуктов.
- Category (Категория): Категория, к которой относится продукт. Продукт может быть одной из следующих категорий: Category 1, Category 2, Category 3.
- Customer ID (Идентификатор клиента): Уникальный идентификатор клиента, совершившего покупку.
- Is Returned (Возврат): Флаг, указывающий, был ли продукт возвращен клиентом. Значение True означает, что продукт был возвращен, а значение False означает, что продукт не был возвращен.

Подготовить отчет по месяцам
 - Число покупок всего
 - Выручка
 - Число возвратов
 - Доля возвратов
 - Выручка без учетов возвратов
 - Сумма возвратов

По результатам отчета найти:
 - Месяц с наибольшим процентом возвратов
 - Месяц с наименьшей выручкой
 
Подготовить отчет по продуктам:
  - сравнить возвраты в срезе по продуктам
  - сравнить выручку в срезе по продуктам
  
Подготовить отчет по покупателям
 - число уникальных покупателей
 - процент покупателей с числом покупок больше 1
 - сохранить все покупки таких покупателей
  

In [1]:
import pandas as pd

In [2]:
sales_data = pd.read_excel('sales_data.xlsx')

In [3]:
sales_data['Date'].max()

Timestamp('2021-12-31 00:00:00')

In [7]:
sales_data

,Product,Date,Quantity,Revenue,Category,Customer ID,Is Returned
0,Product A,2021-08-21,6,73.59,Category 3,4745,not returned
1,Product C,2021-07-24,4,68.88,Category 1,5345,returned
2,Product D,2021-10-30,1,98.15,Category 1,6735,not returned
3,Product C,2021-08-20,10,70.80,Category 2,6370,not returned
4,Product B,2021-07-29,8,75.13,Category 3,9326,not returned
...,...,...,...,...,...,...,...
995,Product C,2021-06-13,9,67.34,Category 3,9996,not returned
996,Product D,2021-02-18,8,68.12,Category 2,7645,returned
997,Product D,2021-10-18,9,11.33,Category 3,7312,not returned
998,Product D,2021-02-24,7,51.35,Category 1,2637,returned


In [5]:
sales_data['Date'].min()

Timestamp('2021-01-01 00:00:00')

In [6]:
sales_data['Is Returned'] = sales_data['Is Returned'].replace([True, False], ['returned', 'not returned'])

In [8]:
sales_data['Month'] = sales_data['Date'].astype('datetime64[M]')

TypeError: Cannot cast DatetimeArray to dtype datetime64[M]

In [18]:
annual_sales_report = sales_data \
    .groupby('Month', as_index=False) \
    .agg(Total_sales = ('Product', 'count'), Total_revenue = ('Revenue', 'sum'))

In [19]:
annual_sales_report

,Month,Total_sales,Total_revenue
0,2021-01-01,76,4197.20
1,2021-02-01,84,4253.56
2,2021-03-01,85,4441.78
3,2021-04-01,83,4440.08
4,2021-05-01,86,4828.20
5,2021-06-01,91,5132.39
6,2021-07-01,74,4218.08
7,2021-08-01,97,4815.18
8,2021-09-01,85,4816.22
9,2021-10-01,76,4357.53


In [24]:
df = pd.pivot_table(sales_data, values='Revenue', index='Month', columns='Is Returned', aggfunc=['sum', 'count'])

In [26]:
df

sum                 count         
Is Returned not returned returned not returned returned
Month                                                  
2021-01-01       1818.56  2378.64           34       42
2021-02-01       1651.37  2602.19           34       50
2021-03-01       2289.94  2151.84           42       43
2021-04-01       2347.98  2092.10           41       42
2021-05-01       2291.16  2537.04           40       46
2021-06-01       2600.69  2531.70           43       48
2021-07-01       2489.30  1728.78           40       34
2021-08-01       2491.73  2323.45           47       50
2021-09-01       2351.99  2464.23           40       45
2021-10-01       2276.80  2080.73           38       38
2021-11-01       2143.66  2629.16           41       46
2021-12-01       1947.17  2178.45           35       41

In [29]:
df['count']['returned'].values

array([42, 50, 43, 42, 46, 48, 34, 50, 45, 38, 46, 41])

In [30]:
annual_sales_report['returns'] = df['count']['returned'].values

In [32]:
annual_sales_report['returned_amount'] = df['sum']['returned'].values

In [34]:
annual_sales_report['returns_percantage'] = annual_sales_report['returns'] / annual_sales_report['Total_sales']

In [35]:
annual_sales_report

,Month,Total_sales,Total_revenue,returns,returned_amount,returns_percantage
0,2021-01-01,76,4197.20,42,2378.64,0.552632
1,2021-02-01,84,4253.56,50,2602.19,0.595238
2,2021-03-01,85,4441.78,43,2151.84,0.505882
3,2021-04-01,83,4440.08,42,2092.10,0.506024
4,2021-05-01,86,4828.20,46,2537.04,0.534884
5,2021-06-01,91,5132.39,48,2531.70,0.527473
6,2021-07-01,74,4218.08,34,1728.78,0.459459
7,2021-08-01,97,4815.18,50,2323.45,0.515464
8,2021-09-01,85,4816.22,45,2464.23,0.529412
9,2021-10-01,76,4357.53,38,2080.73,0.500000


In [38]:
annual_sales_report.sort_values('returns_percantage').tail(1)

,Month,Total_sales,Total_revenue,returns,returned_amount,returns_percantage
1,2021-02-01,84,4253.56,50,2602.19,0.595238


In [40]:
annual_sales_report.sort_values('Total_revenue').head(1)

,Month,Total_sales,Total_revenue,returns,returned_amount,returns_percantage
11,2021-12-01,76,4125.62,41,2178.45,0.539474


In [41]:
sales_data['IsReturned_'] = sales_data['Is Returned'].replace(['returned', 'not returned'], [1, 0])

In [44]:
sales_data.groupby('Product', as_index=False) \
    .agg(return_ratio=('IsReturned_', 'mean'), total_revenue=('Revenue', 'sum'))

,Product,return_ratio,total_revenue
0,Product A,0.502463,11690.93
1,Product B,0.532967,9712.35
2,Product C,0.620370,11593.69
3,Product D,0.460733,10144.77
4,Product E,0.500000,11256.92


In [46]:
sales_data['Customer ID'].nunique()

952

In [47]:
sales_data

,Product,Date,Quantity,Revenue,Category,Customer ID,Is Returned,Month,IsReturned_
0,Product A,2021-08-21,6,73.59,Category 3,4745,not returned,2021-08-01,0
1,Product C,2021-07-24,4,68.88,Category 1,5345,returned,2021-07-01,1
2,Product D,2021-10-30,1,98.15,Category 1,6735,not returned,2021-10-01,0
3,Product C,2021-08-20,10,70.80,Category 2,6370,not returned,2021-08-01,0
4,Product B,2021-07-29,8,75.13,Category 3,9326,not returned,2021-07-01,0
...,...,...,...,...,...,...,...,...,...
995,Product C,2021-06-13,9,67.34,Category 3,9996,not returned,2021-06-01,0
996,Product D,2021-02-18,8,68.12,Category 2,7645,returned,2021-02-01,1
997,Product D,2021-10-18,9,11.33,Category 3,7312,not returned,2021-10-01,0
998,Product D,2021-02-24,7,51.35,Category 1,2637,returned,2021-02-01,1


In [51]:
users_df = sales_data.groupby('Customer ID', as_index=False) \
    .agg(total_products=('Product', 'count'))

In [58]:
users_id = users_df[users_df['total_products'] > 1]['Customer ID']

In [59]:
sales_data[sales_data['Customer ID'].isin(users_id)]

,Product,Date,Quantity,Revenue,Category,Customer ID,Is Returned,Month,IsReturned_
0,Product A,2021-08-21,6,73.59,Category 3,4745,not returned,2021-08-01,0
6,Product E,2021-09-15,8,75.86,Category 2,2877,not returned,2021-09-01,0
10,Product A,2021-10-08,4,18.26,Category 2,8756,returned,2021-10-01,1
11,Product A,2021-03-08,9,28.27,Category 2,5358,not returned,2021-03-01,0
31,Product E,2021-12-18,6,13.59,Category 3,7324,returned,2021-12-01,1
...,...,...,...,...,...,...,...,...,...
947,Product C,2021-12-07,9,62.51,Category 1,7187,not returned,2021-12-01,0
961,Product B,2021-03-23,8,60.21,Category 1,2039,returned,2021-03-01,1
967,Product A,2021-09-08,1,47.87,Category 1,9486,returned,2021-09-01,1
979,Product A,2021-03-02,10,10.32,Category 2,4690,returned,2021-03-01,1
